In [2]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [1]:
pip install onnx

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install skl2onnx

  Using cached skl2onnx-1.17.0-py2.py3-none-any.whl.metadata (3.2 kB)
Using cached skl2onnx-1.17.0-py2.py3-none-any.whl (298 kB)
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install onnxruntime

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install bentoml

Note: you may need to restart the kernel to use updated packages.


In [6]:
import sys

import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split

import onnx

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import time
from collections import Counter
import skl2onnx
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType
import numpy as np
import onnxruntime as ort
from onnx import load
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report,roc_curve, roc_auc_score, f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVC
import bentoml

ImportError: DLL load failed while importing onnx_cpp2py_export: Произошел сбой в программе инициализации библиотеки динамической компоновки (DLL).

In [9]:
data = pd.read_csv('./winequality-red.csv')

In [10]:

y_binary = (data['quality'] >= 6).astype(int)
y_binary.value_counts(normalize=True) * 100

X_fe = data.drop(columns=['quality'])


X_train, X_test, y_train, y_test = train_test_split(X_fe, y_binary, test_size=0.3, random_state=42, stratify=y_binary)

pca = PCA(n_components=8)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

scaler = StandardScaler()  # Создание объекта StandardScaler
X_train = scaler.fit_transform(X_train_pca)  # Обучение и преобразование обучающей выборки
X_test = scaler.transform(X_test_pca)  # Преобразование тестовой выборки

%time model = LogisticRegression(max_iter=200, penalty='l1', solver='liblinear', C=0.5)
%time model.fit(X_train, y_train)

# 6. Прогнозирование на тестовых данных
y_pred = model.predict(X_test)

# 7. Оценка модели
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy:.2f}')
print('Confusion Matrix:')
print(conf_matrix)
print('Classification Report:')
print(class_report)

# 4. Прогнозирование вероятностей для тестовых данных
%time y_probs = model.predict_proba(X_test)[:, 1]  # Вероятности для положительного класса

# 5. Вычисление ROC-кривой
fpr, tpr, thresholds = roc_curve(y_test, y_probs)
roc_auc = roc_auc_score(y_test, y_probs)  # Площадь под ROC-кривой

# 6. Построение ROC-кривой
plt.figure(figsize=(10, 6))
plt.plot(fpr, tpr, color='blue', label='ROC curve (area = {:.2f})'.format(roc_auc))
plt.plot([0, 1], [0, 1], color='red', linestyle='--')  # Линия случайности
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc='lower right')
plt.show()


NameError: name 'PCA' is not defined

In [14]:
pip install onnx onnx-sklearn

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement onnx-sklearn (from versions: none)
ERROR: No matching distribution found for onnx-sklearn


In [15]:
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType

# Установка типа входных данных (размерность должна соответствовать входу вашей модели)
initial_type = [('float_input', FloatTensorType([None, X_train.shape[1]]))]

# Конвертация модели
onnx_model = convert_sklearn(model, initial_types=initial_type)

ImportError: DLL load failed while importing onnx_cpp2py_export: Произошел сбой в программе инициализации библиотеки динамической компоновки (DLL).

In [ ]:
# Определите путь для сохранения
onnx_path = './wine_onnx.onnx'  # Например, если вы сохраняете в текущую директорию

# Сохранение модели
with open('wine_onnx.onnx', "wb") as f:
    f.write(onnx_model.SerializeToString())

In [ ]:
import os

# Проверить, существует ли файл
print(os.path.exists(onnx_path))  # Должно вывести True

True


In [ ]:
initial_type = [('float_input', FloatTensorType([None, X_train.shape[1]]))]
onnx_model = convert_sklearn(model, initial_types=initial_type)

onnx_path = './wine_onnx.onnx'

session = ort.InferenceSession(onnx_path)

input_name = session.get_inputs()[0].name #входной тензор
output_name = session.get_outputs()[0].name #выходной тензор

#Подготовка данных для теста
X_test_np = np.array(X_test, dtype=np.float32)

predict_onnx = session.run([output_name], {input_name: X_test_np})[0]

#Оценка качества
accuracy = accuracy_score(y_test, predict_onnx)
f1 = f1_score(y_test, predict_onnx, average = 'weighted')

print('Accuracy (ONNX):', accuracy)
print("F1-score (ONNX):", f1)
print("Classification Report (ONNX):", classification_report(y_test, predict_onnx))

%time predict_onnx

Accuracy (ONNX): 0.7375
F1-score (ONNX): 0.737828587075575
Classification Report (ONNX):               precision    recall  f1-score   support

           0       0.70      0.75      0.73       223
           1       0.77      0.72      0.75       257

    accuracy                           0.74       480
   macro avg       0.74      0.74      0.74       480
weighted avg       0.74      0.74      0.74       480

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.01 µs


array([1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1,
       1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0,
       1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0,
       1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1,
       0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0,
       1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0,
       1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0,
       1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1,
       0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1,
       0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0,
       0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0,

In [ ]:

# Load your ONNX model
onnx_model = onnx.load("./wine_onnx.onnx")
print(onnx_model)
# Save the model in BentoML
bentoml.onnx.save_model('onnx_model', onnx_model)


class MyONNXService(Service):
    def __init__(self):
        self.model = load_model('onnx_model')

    def predict(self, input_data):
        # Process input_data and make predictions
        return self.model.predict(input_data)


onnx_runner = bentoml.onnx.get('onnx_model').to_runner()

# Определение сервиса
svc = Service(name="onnx_service", runners=[onnx_runner])

# Определение эндпоинта для предсказаний
@svc.api(input=NumpyNdarray(), output=NumpyNdarray())
def predict(input_data: np.ndarray) -> np.ndarray:
    # Передача данных в модель через runner
    return onnx_runner.run(input_data)

ir_version: 10
producer_name: "skl2onnx"
producer_version: "1.17.0"
domain: "ai.onnx"
model_version: 0
doc_string: ""
graph {
  node {
    input: "float_input"
    output: "label"
    output: "probabilities"
    name: "LinearClassifier"
    op_type: "LinearClassifier"
    attribute {
      name: "classlabels_ints"
      ints: 0
      ints: 1
      type: INTS
    }
    attribute {
      name: "coefficients"
      floats: 0.6087689995765686
      floats: -0.29691532254219055
      floats: -0.22102023661136627
      floats: -0.2788827121257782
      floats: -1.0050088167190552
      floats: 0.4859588146209717
      floats: -0.13084676861763
      floats: -0.04924231395125389
      floats: -0.6087689995765686
      floats: 0.29691532254219055
      floats: 0.22102023661136627
      floats: 0.2788827121257782
      floats: 1.0050088167190552
      floats: -0.4859588146209717
      floats: 0.13084676861763
      floats: 0.04924231395125389
      type: FLOATS
    }
    attribute {
      name:

NameError: name 'Service' is not defined

In [ ]:
pip show bentoml

Name: bentoml
Version: 1.3.15
Summary: BentoML: The easiest way to serve AI apps and models
Home-page: https://bentoml.com
Author: 
Author-email: BentoML Team <contact@bentoml.com>
License: Apache-2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: a2wsgi, aiohttp, aiosqlite, attrs, cattrs, circus, click, click-option-group, cloudpickle, deepmerge, fs, httpx, httpx-ws, inflection, jinja2, numpy, nvidia-ml-py, opentelemetry-api, opentelemetry-instrumentation, opentelemetry-instrumentation-aiohttp-client, opentelemetry-instrumentation-asgi, opentelemetry-sdk, opentelemetry-semantic-conventions, opentelemetry-util-http, packaging, pathspec, pip-requirements-parser, prometheus-client, psutil, pydantic, python-dateutil, python-dotenv, python-json-logger, python-multipart, pyyaml, questionary, rich, schema, simple-di, starlette, tomli, tomli-w, uv, uvicorn, watchfiles
Required-by: 


In [ ]:
python -m venv myenv

SyntaxError: invalid syntax (<ipython-input-29-e2850152647f>, line 1)

In [ ]:
import bentoml
from bentoml.io import NumpyNdarray
#from bentoml import artifacts, env
from sklearn.linear_model import LogisticRegression
import numpy as np
import bentoml.BentoService

#@artifacts([bentoml.artifacts.ModelArtifact('model')])
#@env(infer_pip_packages=True)
class LogisticRegressionService(bentoml.BentoService):

    @bentoml.api(input=NumpyNdarray(), output=NumpyNdarray())
    def predict(self, input_data):
        model = self.artifacts.model
        return model.predict(input_data)

if __name__ == "__main__":
    # Подготовка данных
    X = np.random.rand(100, 10)  # 100 образцов, 10 признаков
    y = (X.sum(axis=1) > 5).astype(int)  # Простой бинарный таргет

    # Обучение модели
    model = LogisticRegression()
    model.fit(X, y)

    # Создание экземпляра сервиса и упаковка модели
    service = LogisticRegressionService()
    service.pack('model', model)

    # Сохранение сервиса
    service.save('LogisticRegressionModel')

<ipython-input-26-0e748b6fd900>:6: BentoMLDeprecationWarning: `bentoml.BentoService` is deprecated since BentoML v1.4 and will be removed in a future version.
  import bentoml.BentoService
/usr/lib/python3.10/importlib/util.py:103: BentoMLDeprecationWarning: `bentoml._internal` is deprecated since BentoML v1.4 and will be removed in a future version.
  return _find_spec(fullname, parent_path)


RecursionError: maximum recursion depth exceeded while calling a Python object

In [ ]:
import joblib  # для сохранения модели

# Сохранение модели
joblib.dump(model, 'logistic_model.joblib')




['logistic_model.joblib']

In [ ]:
import bentoml
from bentoml.io import NumpyNdarray
from sklearn.linear_model import LogisticRegression
import joblib
import numpy as np

from bentoml import env, artifacts, BentoService


# Определение BentoService
class LogisticRegressionService(bentoml.BentoService):
    @bentoml.api(input=NumpyNdarray(), output=NumpyNdarray())
    def predict(self, input_data):
       # Загрузка модели
       model = joblib.load('logistic_model.joblib')
       # Предсказание
       return model.predict(input_data)

# Сохранение сервиса
if __name__ == "__main__":
    # Создайте экземпляр сервиса
    service = LogisticRegressionService()
    # Сохраните сервис
    service.save('LogisticRegressionModel')

/usr/local/lib/python3.10/dist-packages/bentoml/io.py:7: BentoMLDeprecationWarning: `bentoml.io` is deprecated since BentoML v1.4 and will be removed in a future version. Please upgrade to new style IO types instead.
  warn_deprecated(
<ipython-input-17-6a868633ff4a>:7: BentoMLDeprecationWarning: `bentoml.env` is deprecated since BentoML v1.4 and will be removed in a future version.
  from bentoml import env, artifacts, BentoService
/usr/lib/python3.10/importlib/util.py:103: BentoMLDeprecationWarning: `bentoml._internal` is deprecated since BentoML v1.4 and will be removed in a future version.
  return _find_spec(fullname, parent_path)


RecursionError: maximum recursion depth exceeded while calling a Python object

In [ ]:
bentoml serve LogisticRegressionModel

In [ ]:
import requests
import numpy as np

# Подготовка данных для отправки (пример с 10 признаками)
data = np.random.rand(1, 10).tolist()  # Создаём случайный пример с 10 признаками

# Отправка POST-запроса
response = requests.post("http://localhost:5000/predict", json=data)

# Вывод результата
print(response.json())  # Покажет предсказание модели

In [ ]:

@bentoml.service(resources={"cpu": "1", "memory": "1Gi"})
class DataModel:
    def __init__(self):
        # Simulating a data analysis model initialization
        self.model = "Simulated data analysis model"

    @bentoml.api
    def process_data(self, data: t.Dict[str, t.Any]) -> t.Dict[str, t.Any]:
        return {"processed_data": data, "status": "success"}



@bentoml.service(resources={"cpu": "4", "memory": "8Gi"})
class InferenceOrchestrator:
    data_processor = bentoml.depends(DataModel)

NameError: name 't' is not defined

In [ ]:
from __future__ import annotations
import bentoml
from transformers import pipeline


EXAMPLE_INPUT = "sometext"

@bentoml.service(
    resources={"cpu": "2"},
    traffic={"timeout": 10},
)

class Model:
    def __init__(self) -> None:
        # Load onnx model and start onn inference service
        self.pipeline = pipeline('summarization')


    @bentoml.api
    def predict(self, input_sample: = EXAMPLE_INPUT) -> str:
# dont forget to add preprocessing  module for input data

        result = self.pipeline(input_sample)

# dont forget to add post processing  module for input data

        return result[0]['summary_text'] #or extract target information

SyntaxError: invalid syntax (<ipython-input-11-e6a21eaa0169>, line 20)

In [3]:
import bentoml

# Save the model in BentoML
bentoml.onnx.save_model('onnx_model', onnx_model)


class MyONNXService(Service):
    def __init__(self):
        self.model = load_model('onnx_model')

    def predict(self, input_data):
        # Process input_data and make predictions
        return self.model.predict(input_data)


onnx_runner = bentoml.onnx.get('onnx_model').to_runner()

# Определение сервиса
svc = Service(name="onnx_service", runners=[onnx_runner])

# Определение эндпоинта для предсказаний
@svc.api(input=NumpyNdarray(), output=NumpyNdarray())
def predict(input_data: np.ndarray) -> np.ndarray:
    # Передача данных в модель через runner
    return onnx_runner.run(input_data)

MissingDependencyException: onnx is required in order to use module 'bentoml.onnx', install onnx with 'pip install onnx'. For more information, refer to https://onnx.ai/get-started.html